In [38]:
import numpy as np
import pandas as pd
import operator
import pickle

In [3]:
## Reading MIMIC Data

data_dir = "../Data/"
drug_codes = pd.read_csv(data_dir+"DRGCODES.csv.gz", compression="gzip")
icu_stays = pd.read_csv(data_dir+"ICUSTAYS.csv.gz", compression="gzip")
#input_events_cv = pd.read_csv(data_dir+"INPUTEVENTS_CV.csv.gz", compression="gzip")
#input_events_mv = pd.read_csv(data_dir+"INPUTEVENTS_MV.csv.gz", compression="gzip")
#lab_events = pd.read_csv(data_dir+"LABEVENTS.csv.gz", compression="gzip")
microbiology_events = pd.read_csv(data_dir+"MICROBIOLOGYEVENTS.csv.gz", compression="gzip")
#note_events = pd.read_csv(data_dir+"NOTEEVENTS.csv.gz", compression="gzip")

In [5]:
#output_events = pd.read_csv(data_dir+"OUTPUTEVENTS.csv.gz", compression="gzip")
patients = pd.read_csv(data_dir+"PATIENTS.csv.gz", compression="gzip")
#prescriptions = pd.read_csv(data_dir+"PRESCRIPTIONS.csv.gz", compression="gzip")
procedure_events_mv = pd.read_csv(data_dir+"PROCEDUREEVENTS_MV.csv.gz", compression="gzip")
procedures_icd = pd.read_csv(data_dir+"PROCEDURES_ICD.csv.gz", compression="gzip")
services = pd.read_csv(data_dir+"SERVICES.csv.gz", compression="gzip")
transfers = pd.read_csv(data_dir+"TRANSFERS.csv.gz", compression="gzip")

In [4]:
prescriptions = pd.read_csv(data_dir+"PRESCRIPTIONS.csv.gz", compression="gzip")
prescriptions.head(10)

/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO
5,2214774,6,107064,NaN,2175-06-11 00:00:00,2175-06-15 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF0,014198,56016975.0,Check with MD for Dose,1,dose,1,dose,PO
6,2215525,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
7,2216266,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
8,2215526,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
9,2214778,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF2,006561,56017075.0,2mg Tab,2,mg,1,TAB,PO


In [6]:
prescriptions["DRUG"] = prescriptions["DRUG"].str.lower()
drugs = prescriptions["DRUG"]
drugs_set = set (drugs)
print (len(drugs_set))
drugs.head(5)

3966


0        tacrolimus
1          warfarin
2    heparin sodium
3               d5w
4        furosemide
Name: DRUG, dtype: object

In [7]:
statins = ['atorvastatin', 'fluvastatin', 'lovastatin', 'pitavastatin', 'pravastatin', 'rosuvastatin', 'simvastatin']
cholestrol_drugs = prescriptions[prescriptions["DRUG"].str.contains("|".join(statins))]["DRUG"]
print(len(cholestrol_drugs))
cholestrol_drugs.head()

30167


352            atorvastatin
476            atorvastatin
719            atorvastatin
764            atorvastatin
896    rosuvastatin calcium
Name: DRUG, dtype: object

In [8]:
statin_count = dict()
for statin in cholestrol_drugs:
    if (statin in statin_count):
        statin_count[statin] += 1
    else:
        statin_count[statin] = 0

In [9]:
statin_count

{'atorvastatin': 17329,
 'atorvastatin study drug': 5,
 'atorvastatin study med': 0,
 'fluvastatin': 0,
 'fluvastatin sodium': 19,
 'lovastatin': 72,
 'pravastatin': 1747,
 'rosuvastatin': 1,
 'rosuvastatin (crestor)': 0,
 'rosuvastatin calcium': 1348,
 'simvastatin': 9603,
 'simvastatin 40 mg or placebo': 0,
 'simvastatin or placebo': 27,
 'simvastatin or placebo ': 0,
 'simvastatin or placebo (*ind*)': 1}

In [9]:
print(len(patients))
patients.head()

46520


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [12]:
cholestrol_prescriptions = prescriptions[prescriptions["DRUG"].str.contains("|".join(statins))]
cholestrol_prescriptions

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
352,2122368,13,143045,NaN,2167-01-08 00:00:00,2167-01-09 00:00:00,MAIN,atorvastatin,Atorvastatin,Atorvastatin,ATOR20,029968,7.101562e+07,20MG TAB,20,mg,1,TAB,PO
476,2123168,13,143045,263738.0,2167-01-12 00:00:00,2167-01-15 00:00:00,MAIN,atorvastatin,Atorvastatin,Atorvastatin,ATOR20,029968,7.101562e+07,20MG TAB,20,mg,1,TAB,PO
719,2489494,24,161859,NaN,2139-06-06 00:00:00,2139-06-09 00:00:00,MAIN,atorvastatin,Atorvastatin,Atorvastatin,ATOR80,045772,7.101582e+07,80mg Tablet,80,mg,1,TAB,PO
764,2349535,25,129635,203487.0,2160-11-02 00:00:00,2160-11-05 00:00:00,MAIN,atorvastatin,Atorvastatin,Atorvastatin,ATOR40,029969,7.101572e+07,40mg Tab,80,mg,2,TAB,PO
896,1426278,18,188822,298129.0,2167-10-02 00:00:00,2167-10-04 00:00:00,MAIN,rosuvastatin calcium,Rosuvastatin Calcium,Rosuvastatin Calcium,ROSU5,052944,3.100756e+08,5mg Tablet,10,mg,2,TAB,PO
1006,1967951,20,157681,264490.0,2183-04-29 00:00:00,2183-05-03 00:00:00,MAIN,atorvastatin,Atorvastatin,Atorvastatin,ATOR20,029968,7.101562e+07,20MG TAB,20,mg,1,TAB,PO
1015,1388984,21,109451,NaN,2134-09-11 00:00:00,2134-09-11 00:00:00,MAIN,simvastatin,Simvastatin,Simvastatin,SIMV40,016579,6.074928e+06,40mg Tablet,80,mg,2,TAB,PO
1058,1388989,21,109451,217847.0,2134-09-12 00:00:00,2134-09-24 00:00:00,MAIN,atorvastatin,Atorvastatin,Atorvastatin,ATOR80,045772,7.101582e+07,80mg Tablet,80,mg,1,TAB,PO
1152,2908573,35,166707,282039.0,2122-02-12 00:00:00,2122-02-15 00:00:00,MAIN,atorvastatin,Atorvastatin,Atorvastatin,ATOR40,029969,7.101572e+07,40mg Tablet,40,mg,1,TAB,PO
1175,2908602,35,166707,282039.0,2122-02-15 00:00:00,2122-02-15 00:00:00,MAIN,atorvastatin,Atorvastatin,Atorvastatin,ATOR40,029969,7.101572e+07,40mg Tablet,40,mg,1,TAB,PO


In [14]:
cholestrol_subjects_set = set(prescriptions[prescriptions["DRUG"].str.contains("|".join(statins))]["SUBJECT_ID"])
len(cholestrol_subjects_set)

15485

In [27]:
cholestrol_drug_subject = dict()
loop_count = 0
for subject in cholestrol_subjects_set:
    loop_count += 1
    cholestrol_drug_subject[subject] = set(prescriptions[(prescriptions["SUBJECT_ID"] == subject) & prescriptions["DRUG"].isin(statins)]["DRUG"])
    if (loop_count % 100 == 0):
        print("Subjects Iterated: ", loop_count)
cholestrol_drug_subject

Subjects Iterated:  100
Subjects Iterated:  200
Subjects Iterated:  300
Subjects Iterated:  400
Subjects Iterated:  500
Subjects Iterated:  600
Subjects Iterated:  700
Subjects Iterated:  800
Subjects Iterated:  900
Subjects Iterated:  1000
Subjects Iterated:  1100
Subjects Iterated:  1200
Subjects Iterated:  1300
Subjects Iterated:  1400
Subjects Iterated:  1500
Subjects Iterated:  1600
Subjects Iterated:  1700
Subjects Iterated:  1800
Subjects Iterated:  1900
Subjects Iterated:  2000
Subjects Iterated:  2100
Subjects Iterated:  2200
Subjects Iterated:  2300
Subjects Iterated:  2400
Subjects Iterated:  2500
Subjects Iterated:  2600
Subjects Iterated:  2700
Subjects Iterated:  2800
Subjects Iterated:  2900
Subjects Iterated:  3000
Subjects Iterated:  3100
Subjects Iterated:  3200
Subjects Iterated:  3300
Subjects Iterated:  3400
Subjects Iterated:  3500
Subjects Iterated:  3600
Subjects Iterated:  3700
Subjects Iterated:  3800
Subjects Iterated:  3900
Subjects Iterated:  4000
Subjects 

{32771: {'simvastatin'},
 98310: {'simvastatin'},
 32777: {'simvastatin'},
 65547: {'atorvastatin'},
 13: {'atorvastatin'},
 32786: {'simvastatin'},
 32787: {'atorvastatin'},
 20: {'atorvastatin'},
 21: {'atorvastatin', 'simvastatin'},
 32790: {'simvastatin'},
 23: {'atorvastatin', 'simvastatin'},
 65560: {'simvastatin'},
 25: {'atorvastatin'},
 26: {'atorvastatin'},
 28: {'pravastatin'},
 32798: {'atorvastatin'},
 98335: {'atorvastatin'},
 98336: {'atorvastatin'},
 34: {'atorvastatin'},
 35: {'atorvastatin'},
 32804: {'simvastatin'},
 32805: {'atorvastatin'},
 38: {'atorvastatin'},
 32807: {'simvastatin'},
 98344: {'pravastatin'},
 65577: {'simvastatin'},
 42: {'atorvastatin'},
 98347: {'atorvastatin', 'simvastatin'},
 44: {'simvastatin'},
 65582: {'atorvastatin', 'simvastatin'},
 5469: {'atorvastatin'},
 49: {'simvastatin'},
 45829: {'atorvastatin'},
 53: {'atorvastatin'},
 55: {'atorvastatin'},
 56: {'atorvastatin'},
 98362: {'atorvastatin'},
 59: {'atorvastatin'},
 98365: {'atorvas

In [28]:
with open('../Data/Cholestrol/cholestrol_drug_subject.pickle', 'wb') as pickler:
    pickle.dump(cholestrol_drug_subject, pickler, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
with open('../Data/Cholestrol/cholestrol_drug_subject.pickle', 'rb') as pickler:
    dict_check = pickle.load(pickler)

print(len(dict_check))

15485


In [31]:
multiple_cholestrol_drug_subjects = dict()
loop_count = 0
for subject in cholestrol_drug_subject:
    loop_count += 1
    if (len(cholestrol_drug_subject[subject]) > 1):
        multiple_cholestrol_drug_subjects[subject] = cholestrol_drug_subject[subject]
    if (loop_count % 100 == 0):
        print("Subjects Iterated: ", loop_count)

with open('../Data/Cholestrol/multiple_cholestrol_drug_subjects.pickle', 'wb') as pickler:
    pickle.dump(multiple_cholestrol_drug_subjects, pickler, protocol=pickle.HIGHEST_PROTOCOL)

Subjects Iterated:  100
Subjects Iterated:  200
Subjects Iterated:  300
Subjects Iterated:  400
Subjects Iterated:  500
Subjects Iterated:  600
Subjects Iterated:  700
Subjects Iterated:  800
Subjects Iterated:  900
Subjects Iterated:  1000
Subjects Iterated:  1100
Subjects Iterated:  1200
Subjects Iterated:  1300
Subjects Iterated:  1400
Subjects Iterated:  1500
Subjects Iterated:  1600
Subjects Iterated:  1700
Subjects Iterated:  1800
Subjects Iterated:  1900
Subjects Iterated:  2000
Subjects Iterated:  2100
Subjects Iterated:  2200
Subjects Iterated:  2300
Subjects Iterated:  2400
Subjects Iterated:  2500
Subjects Iterated:  2600
Subjects Iterated:  2700
Subjects Iterated:  2800
Subjects Iterated:  2900
Subjects Iterated:  3000
Subjects Iterated:  3100
Subjects Iterated:  3200
Subjects Iterated:  3300
Subjects Iterated:  3400
Subjects Iterated:  3500
Subjects Iterated:  3600
Subjects Iterated:  3700
Subjects Iterated:  3800
Subjects Iterated:  3900
Subjects Iterated:  4000
Subjects 

In [36]:
two_drug_subjects = dict()
three_drug_subjects = dict()
for drug in multiple_cholestrol_drug_subjects:
    if (len(multiple_cholestrol_drug_subjects[drug]) == 2):
        two_drug_subjects[drug] =  multiple_cholestrol_drug_subjects[drug]
    elif (len(multiple_cholestrol_drug_subjects[drug]) == 3):
        three_drug_subjects[drug] = multiple_cholestrol_drug_subjects[drug]
print("Total interactions: %i" %len(multiple_cholestrol_drug_subjects))
print("Two drug interactions: %i" %(len(two_drug_subjects)))
print("Three drug interactions: %i" %(len(three_drug_subjects)))

Total interactions: 1291
Two drug interactions: 1251
Three drug interactions: 40


In [39]:
#65719
empty_set = prescriptions[(prescriptions["SUBJECT_ID"] == 65719) & prescriptions["DRUG"].isin(statins)]["DRUG"]
empty_set

Series([], Name: DRUG, dtype: object)